In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
from scipy.stats import iqr
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import roc_curve, auc
from sklearn.datasets import make_classification
from sklearn import metrics, datasets
plt.rcParams['figure.figsize']=(25,5)
DATA_DIRS = "/mnt/a99/d0/sandy/Forecasting/"
df = pd.read_csv(DATA_DIRS+'data/taxi/NYC_Taxi.csv')
data = df[['value']].to_numpy()
labels = np.zeros_like(df[['label']])
# print(sum(labels))
path = "Results/taxi/taxi_ip_336_op_50_predict_50/taxi/"
preds = np.load(path+'trans-mse-ar_pred_mu.npy')
inputs = np.load(path+'inputs.npy')
trues = np.load(path+'targets.npy')
cb = inputs.shape[1]
print(cb,"input length")
preds = preds.reshape(-1,)
trues = trues.reshape(-1,)

# print(preds.shape,inputs.shape,trues.shape)
cwd = "/mnt/a99/d0/sandy/Forecasting/data/"
save_path=os.path.join(cwd,"taxi",f"taxi_mask_roc.npy")
mask_values = np.load(save_path)
print(mask_values.shape,data.shape,labels.shape)
for i,x in enumerate(mask_values):
    if x==1:
        data[i+2*cb]=preds[i]
        labels[i+2*cb]=1
        # print(data[i+200])


336 input length
(9600,) (10320, 1) (10320, 1)


***Below code is for saving predicted anomaly***

In [2]:
df[['label']] = labels
file_p =  os.path.join("/mnt/a99/d0/sandy/Forecasting/data", 'taxi', f'taxi_prediction_roc.csv')
file_ssh =  os.path.join("/mnt/cat/data/sandy/Forecasting", 'data', 'taxi', f'taxi_prediction_roc.csv')
df.to_csv(file_p)
subprocess.run(["scp",file_p, f"sandy@cat.cse.iitb.ac.in:{file_ssh}"])

CompletedProcess(args=['scp', '/mnt/a99/d0/sandy/Forecasting/data/taxi/taxi_prediction_roc.csv', 'sandy@cat.cse.iitb.ac.in:/mnt/cat/data/sandy/Forecasting/data/taxi/taxi_prediction_roc.csv'], returncode=0)

In [ ]:
plt.plot(mask_values*500)
plt.plot(df[2*cb:]['label'].to_numpy()*200)
# plt.plot(preds)
plt.show()

In [ ]:
px.line(df[2*cb:]['value'])

In [ ]:
px.line(data)

**Below code is for imputation data** 

In [10]:
df[['value']] = data
df[['label']] = labels
file_p =  os.path.join("/mnt/a99/d0/sandy/Forecasting/data", 'taxi', f'taxi_imputation_roc.csv')
file_ssh =  os.path.join("/mnt/cat/data/sandy/Forecasting", 'data', 'taxi', f'taxi_imputation_roc.csv')
df.to_csv(file_p)
subprocess.run(["scp",file_p, f"sandy@cat.cse.iitb.ac.in:{file_ssh}"])

CompletedProcess(args=['scp', '/mnt/a99/d0/sandy/Forecasting/data/taxi/taxi_imputation_roc.csv', 'sandy@cat.cse.iitb.ac.in:/mnt/cat/data/sandy/Forecasting/data/taxi/taxi_imputation_roc.csv'], returncode=0)

**Below code show the width of the anomaly predicted**

In [7]:
def series_segmentation(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
outlier_idxs = np.where(df.label == 1)[0]
# outliers = list(data.iloc[outlier_idxs])
outcome = series_segmentation(outlier_idxs)

In [8]:
np.random.seed(34)
mask = np.zeros_like(df['label'])
for line in outcome:
    print(line[-1]-line[0])
print(sum(df['label']),sum(mask))

49
99
199
99
49
49
99
650 0
